# Descubrimientos de patrones de accidentes de tránsito en la CDMX

## Generación de características espaciales directas

In [1]:
from math import radians, cos, sin, asin, sqrt, atan2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from pylab import rcParams
import seaborn as sns
import folium
import networkx as nx
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString, MultiLineString
import shapefile as shp
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
sns.mpl.rc("figure", figsize=(20, 15))
mpl.rcParams['figure.dpi']= 200

### Pre-Procesamiento de Accidentes

In [3]:
url = '/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//ptosC5//pts_real.shp'
pts = gpd.read_file(url)


In [4]:
pts14 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//ptosC5//pts14.shp')
pts15 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//ptosC5//pts15.shp')
pts16 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//ptosC5//pts16.shp')
pts17 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//ptosC5//pts17.shp')
pts18 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//ptosC5//pts18.shp')
pts19 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//ptosC5//pts19.shp')

Agregamos todos los tipos de accidentes por año y el conteo 

In [5]:
lista_a = [pts14,pts15,pts16,pts17,pts18,pts19]
lista_e = ['2014','2015','2016','2017','2018','2019']

list_e =[]
for i,j in zip(lista_a,lista_e):
    tip14 = i.incidente_.value_counts().to_frame().reset_index()
    tip14['ao'] = j
    tip14.columns = ['tipo','conteo','ao']
    list_e.append(tip14)
    
e = pd.concat(list_e)
e.to_csv('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//conteo.csv', index = False)

In [6]:
e

,tipo,conteo,ao
0,accidente-choque sin lesionados,82520,2014
1,accidente-choque con lesionados,27622,2014
2,lesionado-atropellado,25157,2014
3,accidente-volcadura,3766,2014
4,lesionado-accidente automovilístico,1099,2014
5,accidente-persona atrapada / desbarrancada,452,2014
6,accidente-otros,405,2014
7,cadáver-atropellado,343,2014
8,cadáver-accidente automovilístico,143,2014
9,detención ciudadana-atropellado,53,2014


Vemos todos los distintos tipos de accidentes de 2014 a 2019

In [7]:
list(e.tipo.unique())

['accidente-choque sin lesionados',
 'accidente-choque con lesionados',
 'lesionado-atropellado',
 'accidente-volcadura',
 'lesionado-accidente automovilístico',
 'accidente-persona atrapada / desbarrancada',
 'accidente-otros',
 'cadáver-atropellado',
 'cadáver-accidente automovilístico',
 'detención ciudadana-atropellado',
 'detención ciudadana-accidente automovilístico',
 'accidente-ferroviario',
 'accidente-motociclista',
 'accidente-choque con prensados',
 'accidente-vehiculo atrapado',
 'accidente-vehiculo desbarrancado',
 'sismo-choque sin lesionados',
 'sismo-choque con lesionados',
 'sismo-choque con prensados',
 'sismo-persona atropellada',
 'accidente-ciclista',
 'accidente-vehículo atrapado-varado',
 'accidente-monopatín']

Hacemos la homologación de los nombres para reducirlos

In [8]:
di = {'accidente-choque sin lesionados'              :'choque_sin_lesion',
      'sismo-choque sin lesionados'                  :'choque_sin_lesion',
      
      'accidente-choque con lesionados'              :'choque_con_lesion',
      'lesionado-accidente automovilístico'          :'choque_con_lesion',
      'sismo-choque con lesionados'                  :'choque_con_lesion', 
      'detención ciudadana-accidente automovilístico':'choque_con_lesion',
      
      'lesionado-atropellado'                        :'choque_atropellado_grave',
      'detención ciudadana-atropellado'              :'choque_atropellado_grave',
      'sismo-persona atropellada'                    :'choque_atropellado_grave',
      
      'accidente-volcadura'                          :'acc_volcadura_grave', 
       
      'accidente-persona atrapada / desbarrancada'   :'acc_persona_atrapada_grave', 
      
      'accidente-otros'                              :'choque_otros_con_lesion',
      
      'cadáver-atropellado'                          :'atropellado_fatal', 
      
      'cadáver-accidente automovilístico'            :'accidente_fatal',
       
      'accidente-ferroviario'                        :'choque_ferroviario_grave', 
      
      'accidente-motociclista'                       :'choque_motociclista_con_lesion',
      
      'accidente-choque con prensados'               :'choque_prensado_grave', 
      'sismo-choque con prensados'                   :'choque_prensado_grave',
      
      'accidente-vehiculo atrapado'                  :'acc_vehiculo_atrapado_grave',
      'accidente-vehículo atrapado-varado'           :'acc_vehiculo_atrapado_grave',
      
      'accidente-vehiculo desbarrancado'             :'acc_vehiculo_desbarrancado_grave', 
       
      'accidente-ciclista'                           :'choque_ciclista_con_lesion',
      
      'accidente-monopatín'                          :'choque_monopatin_con_lesion'
     }

pts14['incidente_'].replace(di, inplace = True)
pts15['incidente_'].replace(di, inplace = True)
pts16['incidente_'].replace(di, inplace = True)
pts17['incidente_'].replace(di, inplace = True)
pts18['incidente_'].replace(di, inplace = True)
pts19['incidente_'].replace(di, inplace = True)

Por cada año cambiamos los nombres de las columnas de acuerdo al tipo de accidente ocurrido

In [9]:
cat = ['incidente_']
c5_14 = pd.get_dummies(pts14, columns = cat) 
c5_14.columns

Index(['latitud', 'folio', 'ano', 'hora_creac', 'delegacion', 'dia_semana',
       'fecha_crea', 'codigo_cie', 'mes', 'hora_cierr', 'tipo_entra',
       'delegaci_1', 'clas_con_f', 'fecha_cier', 'longitud', 'mesdecierr',
       'geometry', 'incidente__acc_persona_atrapada_grave',
       'incidente__acc_volcadura_grave', 'incidente__accidente_fatal',
       'incidente__atropellado_fatal', 'incidente__choque_atropellado_grave',
       'incidente__choque_con_lesion', 'incidente__choque_ferroviario_grave',
       'incidente__choque_otros_con_lesion', 'incidente__choque_sin_lesion'],
      dtype='object')

In [10]:
c5_14 = c5_14[['geometry', 'ano',
               'incidente__acc_persona_atrapada_grave',
               'incidente__acc_volcadura_grave', 
               'incidente__accidente_fatal',
               'incidente__atropellado_fatal', 
               'incidente__choque_atropellado_grave',
               'incidente__choque_con_lesion', 
               'incidente__choque_ferroviario_grave',
               'incidente__choque_otros_con_lesion', 
               'incidente__choque_sin_lesion']]

c5_14.columns = ['geometry','año',
                 'acc_persona_atrapada_grave',
                 'acc_volcadura_grave', 
                 'accidente_fatal',
                 'atropellado_fatal', 
                 'choque_atropellado_grave',
                 'choque_con_lesion', 
                 'choque_ferroviario_grave',
                 'choque_otros_con_lesion',
                 'choque_sin_lesion']

In [11]:
cat = ['incidente_']
c5_15 = pd.get_dummies(pts15, columns = cat) 
c5_15.columns

Index(['latitud', 'folio', 'ano', 'hora_creac', 'delegacion', 'dia_semana',
       'fecha_crea', 'codigo_cie', 'mes', 'hora_cierr', 'tipo_entra',
       'delegaci_1', 'clas_con_f', 'fecha_cier', 'longitud', 'mesdecierr',
       'geometry', 'incidente__acc_persona_atrapada_grave',
       'incidente__acc_volcadura_grave', 'incidente__accidente_fatal',
       'incidente__atropellado_fatal', 'incidente__choque_atropellado_grave',
       'incidente__choque_con_lesion', 'incidente__choque_otros_con_lesion',
       'incidente__choque_sin_lesion'],
      dtype='object')

In [12]:
c5_15 = c5_15[['geometry', 'ano',
              'incidente__acc_persona_atrapada_grave',
              'incidente__acc_volcadura_grave', 
              'incidente__accidente_fatal',
              'incidente__atropellado_fatal',
              'incidente__choque_atropellado_grave',
              'incidente__choque_con_lesion', 
              'incidente__choque_otros_con_lesion',
              'incidente__choque_sin_lesion']]

c5_15.columns = ['geometry','año',
                 'acc_persona_atrapada_grave',
                 'acc_volcadura_grave', 
                 'accidente_fatal',
                 'atropellado_fatal', 
                 'choque_atropellado_grave',
                 'choque_con_lesion', 
                 'choque_otros_con_lesion',
                 'choque_sin_lesion']

In [13]:
cat = ['incidente_']
c5_16 = pd.get_dummies(pts16, columns = cat) 
c5_16.columns

Index(['latitud', 'folio', 'ano', 'hora_creac', 'delegacion', 'dia_semana',
       'fecha_crea', 'codigo_cie', 'mes', 'hora_cierr', 'tipo_entra',
       'delegaci_1', 'clas_con_f', 'fecha_cier', 'longitud', 'mesdecierr',
       'geometry', 'incidente__acc_persona_atrapada_grave',
       'incidente__acc_volcadura_grave', 'incidente__accidente_fatal',
       'incidente__atropellado_fatal', 'incidente__choque_atropellado_grave',
       'incidente__choque_con_lesion', 'incidente__choque_otros_con_lesion',
       'incidente__choque_sin_lesion'],
      dtype='object')

In [14]:
c5_16 = c5_16[['geometry', 'ano',
               'incidente__acc_persona_atrapada_grave',
               'incidente__acc_volcadura_grave', 
               'incidente__accidente_fatal',
               'incidente__atropellado_fatal', 
               'incidente__choque_atropellado_grave',
               'incidente__choque_con_lesion', 
               'incidente__choque_otros_con_lesion',
               'incidente__choque_sin_lesion']]

c5_16.columns = ['geometry','año',
                 'acc_persona_atrapada_grave',
                 'acc_volcadura_grave', 
                 'accidente_fatal',
                 'atropellado_fatal', 
                 'choque_atropellado_grave',
                 'choque_con_lesion', 
                 'choque_otros_con_lesion',
                 'choque_sin_lesion']

In [15]:
cat = ['incidente_']
c5_17 = pd.get_dummies(pts17, columns = cat) 
c5_17.columns

Index(['latitud', 'folio', 'ano', 'hora_creac', 'delegacion', 'dia_semana',
       'fecha_crea', 'codigo_cie', 'mes', 'hora_cierr', 'tipo_entra',
       'delegaci_1', 'clas_con_f', 'fecha_cier', 'longitud', 'mesdecierr',
       'geometry', 'incidente__acc_persona_atrapada_grave',
       'incidente__acc_vehiculo_atrapado_grave',
       'incidente__acc_vehiculo_desbarrancado_grave',
       'incidente__acc_volcadura_grave', 'incidente__accidente_fatal',
       'incidente__atropellado_fatal', 'incidente__choque_atropellado_grave',
       'incidente__choque_con_lesion', 'incidente__choque_ferroviario_grave',
       'incidente__choque_motociclista_con_lesion',
       'incidente__choque_otros_con_lesion',
       'incidente__choque_prensado_grave', 'incidente__choque_sin_lesion'],
      dtype='object')

In [16]:
c5_17 = c5_17[['geometry', 'ano',
               'incidente__acc_persona_atrapada_grave',
               'incidente__acc_vehiculo_atrapado_grave',
               'incidente__acc_vehiculo_desbarrancado_grave',
               'incidente__acc_volcadura_grave', 
               'incidente__accidente_fatal',
               'incidente__atropellado_fatal', 
               'incidente__choque_atropellado_grave',
               'incidente__choque_con_lesion', 
               'incidente__choque_ferroviario_grave',
               'incidente__choque_motociclista_con_lesion',
               'incidente__choque_otros_con_lesion',
               'incidente__choque_prensado_grave', 
               'incidente__choque_sin_lesion']]

c5_17.columns = ['geometry','año',
                 'acc_persona_atrapada_grave',
                 'acc_vehiculo_atrapado_grave',
                 'acc_vehiculo_desbarrancado_grave',
                 'acc_volcadura_grave', 
                 'accidente_fatal',
                 'atropellado_fatal', 
                 'choque_atropellado_grave',
                 'choque_con_lesion', 
                 'choque_ferroviario_grave',
                 'choque_motociclista_con_lesion',
                 'choque_otros_con_lesion',
                 'choque_prensado_grave', 
                 'choque_sin_lesion']

In [17]:
cat = ['incidente_']
c5_18 = pd.get_dummies(pts18, columns = cat) 
c5_18.columns

Index(['latitud', 'folio', 'ano', 'hora_creac', 'delegacion', 'dia_semana',
       'fecha_crea', 'codigo_cie', 'mes', 'hora_cierr', 'tipo_entra',
       'delegaci_1', 'clas_con_f', 'fecha_cier', 'longitud', 'mesdecierr',
       'geometry', 'incidente__acc_persona_atrapada_grave',
       'incidente__acc_vehiculo_atrapado_grave',
       'incidente__acc_vehiculo_desbarrancado_grave',
       'incidente__acc_volcadura_grave', 'incidente__accidente_fatal',
       'incidente__atropellado_fatal', 'incidente__choque_atropellado_grave',
       'incidente__choque_con_lesion', 'incidente__choque_ferroviario_grave',
       'incidente__choque_motociclista_con_lesion',
       'incidente__choque_otros_con_lesion',
       'incidente__choque_prensado_grave', 'incidente__choque_sin_lesion'],
      dtype='object')

In [18]:
c5_18 = c5_18[['geometry', 'ano',
               'incidente__acc_persona_atrapada_grave',
               'incidente__acc_vehiculo_atrapado_grave',
               'incidente__acc_vehiculo_desbarrancado_grave',
               'incidente__acc_volcadura_grave', 
               'incidente__accidente_fatal',
               'incidente__atropellado_fatal', 
               'incidente__choque_atropellado_grave',
               'incidente__choque_con_lesion', 
               'incidente__choque_ferroviario_grave',
               'incidente__choque_motociclista_con_lesion',
               'incidente__choque_otros_con_lesion',
               'incidente__choque_prensado_grave', 
               'incidente__choque_sin_lesion']]

c5_18.columns = ['geometry','año',
                 'acc_persona_atrapada_grave',
                 'acc_vehiculo_atrapado_grave',
                 'acc_vehiculo_desbarrancado_grave',
                 'acc_volcadura_grave', 
                 'accidente_fatal',
                 'atropellado_fatal', 
                 'choque_atropellado_grave',
                 'choque_con_lesion', 
                 'choque_ferroviario_grave',
                 'choque_motociclista_con_lesion',
                 'choque_otros_con_lesion',
                 'choque_prensado_grave', 
                 'choque_sin_lesion']

In [19]:
cat = ['incidente_']
c5_19 = pd.get_dummies(pts19, columns = cat) 
c5_19.columns

Index(['latitud', 'folio', 'ano', 'hora_creac', 'delegacion', 'dia_semana',
       'fecha_crea', 'codigo_cie', 'mes', 'hora_cierr', 'tipo_entra',
       'delegaci_1', 'clas_con_f', 'fecha_cier', 'longitud', 'mesdecierr',
       'geometry', 'incidente__acc_persona_atrapada_grave',
       'incidente__acc_vehiculo_atrapado_grave',
       'incidente__acc_vehiculo_desbarrancado_grave',
       'incidente__acc_volcadura_grave', 'incidente__accidente_fatal',
       'incidente__atropellado_fatal', 'incidente__choque_atropellado_grave',
       'incidente__choque_ciclista_con_lesion', 'incidente__choque_con_lesion',
       'incidente__choque_ferroviario_grave',
       'incidente__choque_monopatin_con_lesion',
       'incidente__choque_motociclista_con_lesion',
       'incidente__choque_otros_con_lesion',
       'incidente__choque_prensado_grave', 'incidente__choque_sin_lesion'],
      dtype='object')

In [20]:
c5_19 = c5_19[['geometry', 'ano',
               'incidente__acc_persona_atrapada_grave',
               'incidente__acc_vehiculo_atrapado_grave',
               'incidente__acc_vehiculo_desbarrancado_grave',
               'incidente__acc_volcadura_grave', 
               'incidente__accidente_fatal',
               'incidente__atropellado_fatal', 
               'incidente__choque_atropellado_grave',
               'incidente__choque_ciclista_con_lesion', 
               'incidente__choque_con_lesion',
               'incidente__choque_ferroviario_grave',
               'incidente__choque_monopatin_con_lesion',
               'incidente__choque_motociclista_con_lesion',
               'incidente__choque_otros_con_lesion',
               'incidente__choque_prensado_grave', 
               'incidente__choque_sin_lesion']]

c5_19.columns = ['geometry','año',
                 'acc_persona_atrapada_grave',
                 'acc_vehiculo_atrapado_grave',
                 'acc_vehiculo_desbarrancado_grave',
                 'acc_volcadura_grave', 
                 'accidente_fatal',
                 'atropellado_fatal', 
                 'choque_atropellado_grave',
                 'choque_ciclista_con_lesion', 
                 'choque_con_lesion',
                 'choque_ferroviario_grave',
                 'choque_monopatin_con_lesion',
                 'choque_motociclista_con_lesion',
                 'choque_otros_con_lesion',
                 'choque_prensado_grave',
                 'choque_sin_lesion']

Concatenamos todos los años 

In [21]:
# Concatenamos todos los años

df = gpd.GeoDataFrame(pd.concat([c5_14, 
                                 c5_15,
                                 c5_16,
                                 c5_17,
                                 c5_18,
                                 c5_19], ignore_index = True), crs = '4326')
# Convertimos NaNs en 0
df = df.fillna(0)

# Renombramos las columnas
df.columns = ['acc_persona_atrapada_grave', 
              'acc_vehiculo_atrapado_grave',
              'acc_vehiculo_desbarrancado_grave', 
              'acc_volcadura_grave',
              'accidente_fatal', 
              'atropellado_fatal', 
              'año',
              'choque_atropellado_grave', 
              'choque_ciclista_con_lesion',
              'choque_con_lesion', 
              'choque_ferroviario_grave',
              'choque_monopatin_con_lesion', 
              'choque_motociclista_con_lesion',
              'choque_otros_con_lesion', 
              'choque_prensado_grave',
              'choque_sin_lesion',
              'geometry'
             ]




# Funcion que genera la columna por tipo de accidente
def tipo_acc(row):
    if row['acc_persona_atrapada_grave'] != 0: 
        return 'PER_ATRAPADO_DESBARRANCADO'
    elif row['acc_vehiculo_atrapado_grave'] != 0:
        return 'VEH_ATRAPADO'
    elif row['acc_vehiculo_desbarrancado_grave'] != 0:
        return 'VEH_DESBARRANCADO'
    elif row['acc_volcadura_grave'] !=0: 
        return 'VOLCADURA'
    elif row['accidente_fatal'] !=0: 
        return 'ACC_FATAL'
    elif row['atropellado_fatal'] !=0: 
        return 'ATROPELLADO_FATAL'
    elif row['choque_atropellado_grave'] !=0: 
        return 'ATROPELLADO'
    elif row['choque_con_lesion'] !=0: 
        return 'ACC_C_LESION'
    elif row['choque_ferroviario_grave'] !=0: 
        return 'FERROVIARIO' 
    elif row['choque_motociclista_con_lesion'] !=0: 
        return 'MOTOCICLISTA'
    elif row['choque_otros_con_lesion'] !=0: 
        return 'OTROS'
    elif row['choque_prensado_grave'] !=0: 
        return 'ACC_PRENSADO'
    else: 
        return 'ACC_S_LESION'
       
df['TIPO_ACC'] = df.apply(tipo_acc, axis = 1)

# Excluimos los accidentes con ciclista y monopatin
df = df[['año',
         'acc_persona_atrapada_grave', 
         'acc_vehiculo_atrapado_grave',
         'acc_vehiculo_desbarrancado_grave', 
         'acc_volcadura_grave',
         'accidente_fatal', 
         'atropellado_fatal', 
         'choque_atropellado_grave',
         'choque_con_lesion', 
         'choque_ferroviario_grave',
         'choque_motociclista_con_lesion',
         'choque_otros_con_lesion', 
         'choque_prensado_grave',
         'choque_sin_lesion',
         'TIPO_ACC',
         'geometry'
        ]]

# Convertimos la columna de años a variable tipo entero
df.año = df.año.astype(int)

# Nos quedamos con la geometría tipo Punto
points_clean = df[df.geometry.type =='Point']

In [22]:
# Guardamos en formato de shapefile 
points_clean.crs = {'init' :'epsg:4326'}
points_clean.to_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//results_dynamic//dynamic_.shp',encoding = 'utf-8')

In [23]:
# Guardamos en formato CSV
points_clean1 = points_clean.drop('geometry', axis = 1)
points_clean1.to_csv('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//results_dynamic//csv_dynamic_.csv', encoding = 'utf-8', index = False)

In [22]:
points_clean.head()

,año,acc_persona_atrapada_grave,acc_vehiculo_atrapado_grave,acc_vehiculo_desbarrancado_grave,acc_volcadura_grave,accidente_fatal,atropellado_fatal,choque_atropellado_grave,choque_con_lesion,choque_ferroviario_grave,choque_motociclista_con_lesion,choque_otros_con_lesion,choque_prensado_grave,choque_sin_lesion,TIPO_ACC,geometry
0,2014,0,0.0,0.0,0,0,0,0,1,0.0,0.0,0,0.0,0,ACC_C_LESION,POINT (-99.19047999999999 19.43783)
1,2014,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0,0.0,1,ACC_S_LESION,POINT (-99.13297 19.44161)
2,2014,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0,0.0,1,ACC_S_LESION,POINT (-99.19743 19.2289)
3,2014,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0,0.0,1,ACC_S_LESION,POINT (-99.18012 19.4687)
4,2014,0,0.0,0.0,0,0,0,0,1,0.0,0.0,0,0.0,0,ACC_C_LESION,POINT (-99.175695 19.28508)


In [23]:
points_clean.año.unique()

array([2014, 2015, 2016, 2017, 2018, 2019])

### Agregamos los puntos a las mallas hexagonales para el conjunto de entrenamiento

In [25]:
# Cargamos las mallas hexagonales
road_hex50 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//hex_roads//road_hex50.shp', encoding ='utf-8', crs = {'init':'epsg:4326'})
road_hex100 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//hex_roads//road_hex100.shp', encoding ='utf-8', crs = {'init':'epsg:4326'})
road_hex200 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//hex_roads//road_hex200.shp', encoding ='utf-8', crs = {'init':'epsg:4326'})
road_hex300 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//hex_roads//road_hex300.shp', encoding ='utf-8', crs = {'init':'epsg:4326'})
road_hex500 = gpd.read_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1//shp_b//processed_data//hex_roads//road_hex500.shp', encoding ='utf-8', crs = {'init':'epsg:4326'})


In [26]:
road_hex100.head()

,left,bottom,right,top,geometry
0,-99.36541,19.276903,-99.364357,19.277815,"POLYGON ((-99.36541 19.27735919863746, -99.365..."
1,-99.36541,19.277815,-99.364357,19.278728,"POLYGON ((-99.36541 19.2782714124881, -99.3651..."
2,-99.36462,19.276447,-99.363567,19.277359,POLYGON ((-99.36461999963166 19.27690309171214...
3,-99.36462,19.277359,-99.363567,19.278271,POLYGON ((-99.36461999963166 19.27781530556278...
4,-99.36383,19.276903,-99.362777,19.277815,POLYGON ((-99.36382999926332 19.27735919863746...


In [27]:
road_hex50 = road_hex50.reset_index()
road_hex50.rename(columns={'index':'FID'}, inplace = True)

road_hex100 = road_hex100.reset_index()
road_hex100.rename(columns={'index':'FID'}, inplace = True)

road_hex200 = road_hex200.reset_index()
road_hex200.rename(columns={'index':'FID'}, inplace = True)

road_hex300 = road_hex300.reset_index()
road_hex300.rename(columns={'index':'FID'}, inplace = True)

road_hex500 = road_hex500.reset_index()
road_hex500.rename(columns={'index':'FID'}, inplace = True)


Función para la agregación de las variables directas a la malla, donde:
    1. g --> es la malla a la que se agregaran las características directas.
    2. s --> es la característica directa que se agregará.
    3. Y --> es el tipo de característica directa.
    4. D1--> es el año inicial del rango a filtrar.
    5. D2--> es el año final del rango a filtrar.

In [28]:
def agreg_dir(g, s, Y, D1, D2):
    s = s[(s.año >= D1) & (s.año <= D2)]
    s = s[s.TIPO_ACC == Y]
    s = s[['geometry']].copy()
    s = gpd.sjoin(g, s, how='left', op='intersects')
    s = s.fillna(0)
    s = s.rename(columns={'index_right': Y})
    s[Y] = s[Y].apply(lambda x: 0 if x == 0.0 else 1)
    gs = s.groupby('FID', as_index=False).agg({Y: 'sum',
                                           'geometry': 'first'})
    gs = gpd.GeoDataFrame(gs, crs='4326')
    print("Característica directa agregada",Y,gs.shape)
    return gs

In [33]:
s  = points_clean
g  = road_hex100
D1 = 2014
D2 = 2018

In [ ]:

lst_acc = ['PER_ATRAPADO_DESBARRANCADO',
           'VEH_ATRAPADO',
           'VEH_DESBARRANCADO',
           'VOLCADURA',
           'ACC_FATAL',
           'ATROPELLADO_FATAL',
           'ATROPELLADO',
           'ACC_C_LESION',
           'FERROVIARIO',
           'MOTOCICLISTA',
           'OTROS',
           'ACC_PRENSADO',
           'ACC_S_LESION'
          ]

lst_a = []
for Y in lst_acc:
    e = agreg_dir(g, s, Y, D1, D2)
    lst_a.append(e)
    
    
    
PER_ATRAPADO_DESBARRANCADO = lst_a[0]
VEH_ATRAPADO               = lst_a[1]
VEH_DESBARRANCADO          = lst_a[2]
VOLCADURA                  = lst_a[3]
ACC_FATAL                  = lst_a[4]
ATROPELLADO_FATAL          = lst_a[5]
ATROPELLADO                = lst_a[6]
ACC_C_LESION               = lst_a[7]
FERROVIARIO                = lst_a[8]
MOTOCICLISTA               = lst_a[9]
OTROS                      = lst_a[10]
ACC_PRENSADO               = lst_a[11]
ACC_S_LESION               = lst_a[12]
    
    


Función para la agregación de las variables directas a la malla, con buffer donde:
    1. g --> es la malla a la que se agregaran las características directas.
    2. s --> es la característica directa que se agregará.
    3. Y --> es el tipo de característica directa.
    4. D1--> es el año inicial del rango a filtrar.
    5. D2--> es el año final del rango a filtrar.
    6. m --> son los metros de buffer a partir de cada punto.

In [48]:
def agreg_dir_dist_fecha(g, s, Y, D1, D2, m): 
    s = s[(s.año >= D1) & (s.año <= D2)]
    s = s[s.TIPO_ACC == Y]
    s = s[['geometry']].copy()
    dist = (m*0.1)/11000
    s['geometry']= s.geometry.buffer(dist)
    s = gpd.sjoin(g, s, how='left', op='intersects')
    s = s.fillna(0)
    Y = Y+'_'+str(m)
    s = s.rename(columns={'index_right': Y})
    s[Y] = s[Y].apply(lambda x: 0 if x == 0.0 else 1)
    gs = s.groupby('FID', as_index=False).agg({Y: 'sum',
                                            'geometry': 'first'})
    gs = gpd.GeoDataFrame(gs, crs='4326')
    print("Característica directa agregada",Y,gs.shape)
    return gs

In [83]:
lst_acc = ['PER_ATRAPADO_DESBARRANCADO',
           'VEH_ATRAPADO',
           'VEH_DESBARRANCADO',
           'VOLCADURA',
           'ACC_FATAL',
           'ATROPELLADO_FATAL',
           'ATROPELLADO',
           'ACC_C_LESION',
           'FERROVIARIO',
           'MOTOCICLISTA',
           'OTROS',
           'ACC_PRENSADO',
           'ACC_S_LESION'
          ]

m = 200

lst_a = []
for Y in lst_acc:
    e = agreg_dir_dist_fecha(g, s, Y, D1, D2,m)
    lst_a.append(e)
    
    
    
PER_ATRAPADO_DESBARRANCADO_200 = lst_a[0]
VEH_ATRAPADO_200               = lst_a[1]
VEH_DESBARRANCADO_200          = lst_a[2]
VOLCADURA_200                  = lst_a[3]
ACC_FATAL_200                  = lst_a[4]
ATROPELLADO_FATAL_200          = lst_a[5]
ATROPELLADO_200                = lst_a[6]
ACC_C_LESION_200               = lst_a[7]
FERROVIARIO_200                = lst_a[8]
MOTOCICLISTA_200               = lst_a[9]
OTROS_200                      = lst_a[10]
ACC_PRENSADO_200               = lst_a[11]
ACC_S_LESION_200               = lst_a[12]

Característica directa agregada PER_ATRAPADO_DESBARRANCADO_200 (25859, 3)
Característica directa agregada VEH_ATRAPADO_200 (25859, 3)
Característica directa agregada VEH_DESBARRANCADO_200 (25859, 3)
Característica directa agregada VOLCADURA_200 (25859, 3)
Característica directa agregada ACC_FATAL_200 (25859, 3)
Característica directa agregada ATROPELLADO_FATAL_200 (25859, 3)
Característica directa agregada ATROPELLADO_200 (25859, 3)
Característica directa agregada ACC_C_LESION_200 (25859, 3)
Característica directa agregada FERROVIARIO_200 (25859, 3)
Característica directa agregada MOTOCICLISTA_200 (25859, 3)
Característica directa agregada OTROS_200 (25859, 3)
Característica directa agregada ACC_PRENSADO_200 (25859, 3)
Característica directa agregada ACC_S_LESION_200 (25859, 3)


In [84]:
def concat_dataset(df1, df2):
    data = pd.concat([df1, df2], axis = 1)
    _, i = np.unique(data.columns, return_index=True)
    data = data.iloc[:, i]
    return data

In [85]:
data = g.copy()
data["X"] = data.centroid.x
data["Y"] = data.centroid.y

In [86]:
data = concat_dataset(data, PER_ATRAPADO_DESBARRANCADO)
data = concat_dataset(data, VEH_ATRAPADO)              
data = concat_dataset(data, VEH_DESBARRANCADO)         
data = concat_dataset(data, VOLCADURA)                 
data = concat_dataset(data, ACC_FATAL)                 
data = concat_dataset(data, ATROPELLADO_FATAL)         
data = concat_dataset(data, ATROPELLADO)               
data = concat_dataset(data, ACC_C_LESION)              
data = concat_dataset(data, FERROVIARIO)               
data = concat_dataset(data, MOTOCICLISTA)              
data = concat_dataset(data, OTROS)                     
data = concat_dataset(data, ACC_PRENSADO)              
data = concat_dataset(data, ACC_S_LESION)              

data = concat_dataset(data, PER_ATRAPADO_DESBARRANCADO_100)
data = concat_dataset(data, VEH_ATRAPADO_100)              
data = concat_dataset(data, VEH_DESBARRANCADO_100)         
data = concat_dataset(data, VOLCADURA_100)                 
data = concat_dataset(data, ACC_FATAL_100)                 
data = concat_dataset(data, ATROPELLADO_FATAL_100)         
data = concat_dataset(data, ATROPELLADO_100)               
data = concat_dataset(data, ACC_C_LESION_100)              
data = concat_dataset(data, FERROVIARIO_100)               
data = concat_dataset(data, MOTOCICLISTA_100)              
data = concat_dataset(data, OTROS_100)                     
data = concat_dataset(data, ACC_PRENSADO_100) 
data = concat_dataset(data, ACC_S_LESION_100)

data = concat_dataset(data, PER_ATRAPADO_DESBARRANCADO_200)
data = concat_dataset(data, VEH_ATRAPADO_200)              
data = concat_dataset(data, VEH_DESBARRANCADO_200)         
data = concat_dataset(data, VOLCADURA_200)                 
data = concat_dataset(data, ACC_FATAL_200)                 
data = concat_dataset(data, ATROPELLADO_FATAL_200)         
data = concat_dataset(data, ATROPELLADO_200)               
data = concat_dataset(data, ACC_C_LESION_200)              
data = concat_dataset(data, FERROVIARIO_200)               
data = concat_dataset(data, MOTOCICLISTA_200)              
data = concat_dataset(data, OTROS_200)                     
data = concat_dataset(data, ACC_PRENSADO_200)              
data = concat_dataset(data, ACC_S_LESION_200)

In [87]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25859 entries, 0 to 25858
Data columns (total 47 columns):
ACC_C_LESION                      25859 non-null int64
ACC_C_LESION_100                  25859 non-null int64
ACC_C_LESION_200                  25859 non-null int64
ACC_FATAL                         25859 non-null int64
ACC_FATAL_100                     25859 non-null int64
ACC_FATAL_200                     25859 non-null int64
ACC_PRENSADO                      25859 non-null int64
ACC_PRENSADO_100                  25859 non-null int64
ACC_PRENSADO_200                  25859 non-null int64
ACC_S_LESION                      25859 non-null int64
ACC_S_LESION_100                  25859 non-null int64
ACC_S_LESION_200                  25859 non-null int64
ATROPELLADO                       25859 non-null int64
ATROPELLADO_100                   25859 non-null int64
ATROPELLADO_200                   25859 non-null int64
ATROPELLADO_FATAL                 25859 non-null int64
ATR

In [88]:
data = data[['X','Y','ACC_C_LESION', 'ACC_C_LESION_100', 'ACC_C_LESION_200', 'ACC_FATAL','ACC_FATAL_100', 
             'ACC_FATAL_200','ACC_PRENSADO','ACC_PRENSADO_100','ACC_PRENSADO_200','ACC_S_LESION','ACC_S_LESION_100',
             'ACC_S_LESION_200', 'ATROPELLADO','ATROPELLADO_100','ATROPELLADO_200','ATROPELLADO_FATAL',
             'ATROPELLADO_FATAL_100','ATROPELLADO_FATAL_200','FERROVIARIO','FERROVIARIO_100','FERROVIARIO_200',
             'MOTOCICLISTA', 'MOTOCICLISTA_100','MOTOCICLISTA_200', 'OTROS', 'OTROS_100','OTROS_200',
             'PER_ATRAPADO_DESBARRANCADO', 'PER_ATRAPADO_DESBARRANCADO_100','PER_ATRAPADO_DESBARRANCADO_200',
             'VEH_ATRAPADO','VEH_ATRAPADO_100','VEH_ATRAPADO_200','VEH_DESBARRANCADO','VEH_DESBARRANCADO_100',
             'VEH_DESBARRANCADO_200','VOLCADURA','VOLCADURA_100','VOLCADURA_200','geometry']]


In [89]:
data.crs = {'init':'epsg:4326'}
data.to_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1/results_dynamic/geo_200_train_data.shp')

In [90]:
dt = data.copy()
del dt['geometry']
dt.to_csv('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1/results_dynamic/geo_200_train_data.csv')

### Agregamos los puntos a las mallas hexagonales para el conjunto de prueba

In [51]:
s  = points_clean
g  = road_hex100
D1 = 2015
D2 = 2015

In [ ]:
lst_acc = ['PER_ATRAPADO_DESBARRANCADO',
           'VEH_ATRAPADO',
           'VEH_DESBARRANCADO',
           'VOLCADURA',
           'ACC_FATAL',
           'ATROPELLADO_FATAL',
           'ATROPELLADO',
           'ACC_C_LESION',
           'FERROVIARIO',
           'MOTOCICLISTA',
           'OTROS',
           'ACC_PRENSADO',
           'ACC_S_LESION'
          ]

lst_a = []
for Y in lst_acc:
    e = agreg_dir(g, s, Y, D1, D2)
    lst_a.append(e)
    
    
    
PER_ATRAPADO_DESBARRANCADO = lst_a[0]
VEH_ATRAPADO               = lst_a[1]
VEH_DESBARRANCADO          = lst_a[2]
VOLCADURA                  = lst_a[3]
ACC_FATAL                  = lst_a[4]
ATROPELLADO_FATAL          = lst_a[5]
ATROPELLADO                = lst_a[6]
ACC_C_LESION               = lst_a[7]
FERROVIARIO                = lst_a[8]
MOTOCICLISTA               = lst_a[9]
OTROS                      = lst_a[10]
ACC_PRENSADO               = lst_a[11]
ACC_S_LESION               = lst_a[12]

In [94]:
lst_acc = ['PER_ATRAPADO_DESBARRANCADO',
           'VEH_ATRAPADO',
           'VEH_DESBARRANCADO',
           'VOLCADURA',
           'ACC_FATAL',
           'ATROPELLADO_FATAL',
           'ATROPELLADO',
           'ACC_C_LESION',
           'FERROVIARIO',
           'MOTOCICLISTA',
           'OTROS',
           'ACC_PRENSADO',
           'ACC_S_LESION'
          ]

m = 200

lst_a = []
for Y in lst_acc:
    e = agreg_dir_dist_fecha(g, s, Y, D1, D2,m)
    lst_a.append(e)
    
    
    
PER_ATRAPADO_DESBARRANCADO_200 = lst_a[0]
VEH_ATRAPADO_200               = lst_a[1]
VEH_DESBARRANCADO_200          = lst_a[2]
VOLCADURA_200                  = lst_a[3]
ACC_FATAL_200                  = lst_a[4]
ATROPELLADO_FATAL_200          = lst_a[5]
ATROPELLADO_200                = lst_a[6]
ACC_C_LESION_200               = lst_a[7]
FERROVIARIO_200                = lst_a[8]
MOTOCICLISTA_200               = lst_a[9]
OTROS_200                      = lst_a[10]
ACC_PRENSADO_200               = lst_a[11]
ACC_S_LESION_200               = lst_a[12]

Característica directa agregada PER_ATRAPADO_DESBARRANCADO_200 (25859, 3)
Característica directa agregada VEH_ATRAPADO_200 (25859, 3)
Característica directa agregada VEH_DESBARRANCADO_200 (25859, 3)
Característica directa agregada VOLCADURA_200 (25859, 3)
Característica directa agregada ACC_FATAL_200 (25859, 3)
Característica directa agregada ATROPELLADO_FATAL_200 (25859, 3)
Característica directa agregada ATROPELLADO_200 (25859, 3)
Característica directa agregada ACC_C_LESION_200 (25859, 3)
Característica directa agregada FERROVIARIO_200 (25859, 3)
Característica directa agregada MOTOCICLISTA_200 (25859, 3)
Característica directa agregada OTROS_200 (25859, 3)
Característica directa agregada ACC_PRENSADO_200 (25859, 3)
Característica directa agregada ACC_S_LESION_200 (25859, 3)


In [95]:
data = g.copy()
data["X"] = data.centroid.x
data["Y"] = data.centroid.y

In [96]:
data = concat_dataset(data, PER_ATRAPADO_DESBARRANCADO)
data = concat_dataset(data, VEH_ATRAPADO)              
data = concat_dataset(data, VEH_DESBARRANCADO)         
data = concat_dataset(data, VOLCADURA)                 
data = concat_dataset(data, ACC_FATAL)                 
data = concat_dataset(data, ATROPELLADO_FATAL)         
data = concat_dataset(data, ATROPELLADO)               
data = concat_dataset(data, ACC_C_LESION)              
data = concat_dataset(data, FERROVIARIO)               
data = concat_dataset(data, MOTOCICLISTA)              
data = concat_dataset(data, OTROS)                     
data = concat_dataset(data, ACC_PRENSADO)              
data = concat_dataset(data, ACC_S_LESION)       

data = concat_dataset(data, PER_ATRAPADO_DESBARRANCADO_100)
data = concat_dataset(data, VEH_ATRAPADO_100)              
data = concat_dataset(data, VEH_DESBARRANCADO_100)         
data = concat_dataset(data, VOLCADURA_100)                 
data = concat_dataset(data, ACC_FATAL_100)                 
data = concat_dataset(data, ATROPELLADO_FATAL_100)         
data = concat_dataset(data, ATROPELLADO_100)               
data = concat_dataset(data, ACC_C_LESION_100)              
data = concat_dataset(data, FERROVIARIO_100)               
data = concat_dataset(data, MOTOCICLISTA_100)              
data = concat_dataset(data, OTROS_100)                     
data = concat_dataset(data, ACC_PRENSADO_100)  
data = concat_dataset(data, ACC_S_LESION_100)

data = concat_dataset(data, PER_ATRAPADO_DESBARRANCADO_200)
data = concat_dataset(data, VEH_ATRAPADO_200)              
data = concat_dataset(data, VEH_DESBARRANCADO_200)         
data = concat_dataset(data, VOLCADURA_200)                 
data = concat_dataset(data, ACC_FATAL_200)                 
data = concat_dataset(data, ATROPELLADO_FATAL_200)         
data = concat_dataset(data, ATROPELLADO_200)               
data = concat_dataset(data, ACC_C_LESION_200)              
data = concat_dataset(data, FERROVIARIO_200)               
data = concat_dataset(data, MOTOCICLISTA_200)              
data = concat_dataset(data, OTROS_200)                     
data = concat_dataset(data, ACC_PRENSADO_200)              
data = concat_dataset(data, ACC_S_LESION_200)

In [97]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25859 entries, 0 to 25858
Data columns (total 47 columns):
ACC_C_LESION                      25859 non-null int64
ACC_C_LESION_100                  25859 non-null int64
ACC_C_LESION_200                  25859 non-null int64
ACC_FATAL                         25859 non-null int64
ACC_FATAL_100                     25859 non-null int64
ACC_FATAL_200                     25859 non-null int64
ACC_PRENSADO                      25859 non-null int64
ACC_PRENSADO_100                  25859 non-null int64
ACC_PRENSADO_200                  25859 non-null int64
ACC_S_LESION                      25859 non-null int64
ACC_S_LESION_100                  25859 non-null int64
ACC_S_LESION_200                  25859 non-null int64
ATROPELLADO                       25859 non-null int64
ATROPELLADO_100                   25859 non-null int64
ATROPELLADO_200                   25859 non-null int64
ATROPELLADO_FATAL                 25859 non-null int64
ATR

In [98]:
data = data[['X','Y','ACC_C_LESION', 'ACC_C_LESION_100', 'ACC_C_LESION_200', 'ACC_FATAL','ACC_FATAL_100', 
             'ACC_FATAL_200','ACC_PRENSADO','ACC_PRENSADO_100','ACC_PRENSADO_200','ACC_S_LESION','ACC_S_LESION_100', 
             'ACC_S_LESION_200','ATROPELLADO','ATROPELLADO_100','ATROPELLADO_200','ATROPELLADO_FATAL','ATROPELLADO_FATAL_100',
             'ATROPELLADO_FATAL_200','FERROVIARIO','FERROVIARIO_100','FERROVIARIO_200','MOTOCICLISTA', 
             'MOTOCICLISTA_100','MOTOCICLISTA_200', 'OTROS', 'OTROS_100','OTROS_200','PER_ATRAPADO_DESBARRANCADO', 
             'PER_ATRAPADO_DESBARRANCADO_100','PER_ATRAPADO_DESBARRANCADO_200','VEH_ATRAPADO','VEH_ATRAPADO_100',
             'VEH_ATRAPADO_200','VEH_DESBARRANCADO','VEH_DESBARRANCADO_100','VEH_DESBARRANCADO_200','VOLCADURA',
             'VOLCADURA_100','VOLCADURA_200','geometry']]

In [99]:
data.crs = {'init':'epsg:4326'}
data.to_file('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1/results_dynamic/geo_200_test_data.shp')

In [100]:
dt = data.copy()
del dt['geometry']
dt.to_csv('/Users/daniel.rodriguez/Documents/ACC/ACC_PROOF/ACC1/results_dynamic/geo_200_test_data.csv')